In [41]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from geopy.geocoders import Nominatim
from tqdm import tqdm
import re

In [2]:
rest1 = pd.read_csv('restaurantes-mad.csv')

In [3]:
rest1.head()

,_id,enlace,nombre,puntuacion,tipo_cocina,ubicacion
0,Spicy Soul Hot Pot,http://blog.malawaimai.com/,Spicy Soul Hot Pot,"5,0",['China'],"Calle de Diego de León 11, 28006 Madrid España"
1,Los Montes de Galicia,http://www.losmontesdegalicia.es/,Los Montes de Galicia,"5,0","['€€ - €€€', 'Internacional', 'Mediterránea', ...","Calle Azcona 46, 28028 Madrid España"
2,Empanadas Malvón,http://www.malvon.es/sucursal/malvon-luchana,Empanadas Malvón,"5,0","['Argentina', 'Sudamericana']","Calle de Luchana 3, 28010 Madrid España"
3,Pastamore,http://pastamore.es/,Pastamore,"5,0","['€€ - €€€', 'Italiana', 'Pizza', 'Mediterránea']","Calle Alcalá, 191, 28009 Madrid España"
4,Amazonia Brunch And Cocktail,No hay link,Amazonia Brunch And Cocktail,"5,0","['€€ - €€€', 'Americana', 'Café', 'Internacion...","Calle del Pez 4, 28004 Madrid España"


In [4]:
rest1.head()

,_id,enlace,nombre,puntuacion,tipo_cocina,ubicacion
0,Spicy Soul Hot Pot,http://blog.malawaimai.com/,Spicy Soul Hot Pot,"5,0",['China'],"Calle de Diego de León 11, 28006 Madrid España"
1,Los Montes de Galicia,http://www.losmontesdegalicia.es/,Los Montes de Galicia,"5,0","['€€ - €€€', 'Internacional', 'Mediterránea', ...","Calle Azcona 46, 28028 Madrid España"
2,Empanadas Malvón,http://www.malvon.es/sucursal/malvon-luchana,Empanadas Malvón,"5,0","['Argentina', 'Sudamericana']","Calle de Luchana 3, 28010 Madrid España"
3,Pastamore,http://pastamore.es/,Pastamore,"5,0","['€€ - €€€', 'Italiana', 'Pizza', 'Mediterránea']","Calle Alcalá, 191, 28009 Madrid España"
4,Amazonia Brunch And Cocktail,No hay link,Amazonia Brunch And Cocktail,"5,0","['€€ - €€€', 'Americana', 'Café', 'Internacion...","Calle del Pez 4, 28004 Madrid España"


In [5]:
len_id = list(range(1, (len(rest1['_id'])+1)))

In [6]:
rest1._id = len_id

In [7]:
pais = []
ciudad = []
cp = []
direccion = []

for e in rest1.ubicacion:
    pais.append(e[-6::])
    ciudad.append(e[-13:-7:])
    cp.append(e[-19:-14:])
    direccion.append(e[:-21:])

In [8]:
rest1['pais'] = pais
rest1['ciudad'] = ciudad
rest1['cp'] = cp
rest1['direccion'] = direccion

In [9]:
rest1.head()

,_id,enlace,nombre,puntuacion,tipo_cocina,ubicacion,pais,ciudad,cp,direccion
0,1,http://blog.malawaimai.com/,Spicy Soul Hot Pot,"5,0",['China'],"Calle de Diego de León 11, 28006 Madrid España",España,Madrid,28006,Calle de Diego de León 11
1,2,http://www.losmontesdegalicia.es/,Los Montes de Galicia,"5,0","['€€ - €€€', 'Internacional', 'Mediterránea', ...","Calle Azcona 46, 28028 Madrid España",España,Madrid,28028,Calle Azcona 46
2,3,http://www.malvon.es/sucursal/malvon-luchana,Empanadas Malvón,"5,0","['Argentina', 'Sudamericana']","Calle de Luchana 3, 28010 Madrid España",España,Madrid,28010,Calle de Luchana 3
3,4,http://pastamore.es/,Pastamore,"5,0","['€€ - €€€', 'Italiana', 'Pizza', 'Mediterránea']","Calle Alcalá, 191, 28009 Madrid España",España,Madrid,28009,"Calle Alcalá, 191"
4,5,No hay link,Amazonia Brunch And Cocktail,"5,0","['€€ - €€€', 'Americana', 'Café', 'Internacion...","Calle del Pez 4, 28004 Madrid España",España,Madrid,28004,Calle del Pez 4


In [10]:
rest1 = rest1.reindex( columns =['_id', 'nombre', 'puntuacion','direccion', 'cp', 'ciudad', 'pais', 'enlace','tipo_cocina'])

In [11]:
rest1['direccion'] = rest1['direccion'].apply(lambda x: 'Avenida de la Hispanidad s/n' if 'Madrid-Barajas' in x else x)

In [12]:
rest1['direccion'] = rest1['direccion'].apply(lambda x: 'Avenida de la Hispanidad s/n' if 'Terminal' in x else x)

In [74]:
rest1.loc[5, 'direccion'] = 'Calle de Mejía Lequerica, 10'
rest1.loc[30, 'direccion'] = 'Paseo del Pintor Rosales, 52'
rest1.loc[61, 'direccion'] = 'Calle de Juan Bravo, 43'
rest1.loc[143, 'direccion'] = 'Calle de San Bernardo, 10'
rest1.loc[236, 'direccion'] = 'Calle de Francisco Silvela, 83'
rest1.loc[260, 'direccion'] = 'Calle de la Ballesta 13'
rest1.loc[390, 'direccion'] = 'Calle de Ventura de la Vega, 15'
rest1.loc[424, 'direccion'] = 'Paseo de la Castellana, 259B'
rest1.loc[530, 'direccion'] = 'Calle de San Marcos, 37'
rest1.loc[560, 'direccion'] = 'Calle de Jose Antonio de Armona, 7'
rest1.loc[576, 'direccion'] = 'Calle de Espoz y Mina, 9'
rest1.loc[632, 'direccion'] = 'Calle de San Bernardo, 85'
rest1.loc[638, 'direccion'] = 'Paseo de la Castellana, 57'
rest1.loc[887, 'direccion'] = 'Calle General Ramírez de Madrid, 16'
rest1.loc[994, 'direccion'] = 'Calle de Torija, 7'
rest1.loc[1093, 'direccion'] = 'Calle de Canarias, 63'
rest1.loc[1098, 'direccion'] = 'Paseo del Prado, 48'
rest1.loc[1228, 'direccion'] = 'Carretera del Colmenar Viejo Km 14.5'
rest1.loc[1266, 'direccion'] = 'Calle de Bretón de los Herreos, 58'
rest1.loc[1339, 'direccion'] = 'Calle de Cochabamba, 13'
rest1.loc[1349, 'direccion'] = 'Calle de Isabel la Católica, 11'
rest1.loc[1723, 'direccion'] = 'Calle de Ibiza, 8'
rest1.loc[1772, 'direccion'] = 'Plaza de los Olivos, 1'
rest1.loc[1775, 'direccion'] =  'Calle del Príncipe de Anglona, 1'
rest1.loc[1775, 'cp'] = 28005
rest1.loc[1875, 'direccion'] = 'Calle Padilla, 4'
rest1.loc[2044, 'direccion'] = 'Calle de Serrano Jover, 6'
rest1.loc[2123, 'direccion'] = 'Calle de Toques, 31'
rest1.loc[2137, 'direccion'] = 'Calle Espoz y Mina, 28'
rest1.loc[2180, 'direccion'] =  'Plaza de España, 18'
rest1.loc[2206, 'direccion'] = 'Plaza de San Miguel'
rest1.loc[2250, 'direccion'] = 'Calle de Fernando VI, 21'
rest1.loc[2277, 'direccion'] = 'Avenida de Logroño, 205'
rest1.loc[2278, 'direccion'] = 'Calle Iglesia, 4'
rest1.loc[2278, 'cp'] = 28189
rest1.loc[2400, 'direccion'] = 'Avenida de Portugal, 1'
rest1.loc[2490, 'direccion'] = 'Calle de San Agustin, 6'
rest1.loc[2620, 'direccion'] = 'Calle del Monasterio de las Batuecas, 5'
rest1.loc[2635, 'direccion'] = 'Calle General Ramirez de Madrid, 14'
rest1.loc[2665, 'direccion'] = 'Avenida de la Hispanidad s/n'
rest1.loc[2665, 'cp'] = 28042
rest1.loc[6969, 'cp'] = 28042
rest1.loc[2741, 'direccion'] = 'Calle Raimundo Fernández Villaverde, 65'
rest1.loc[2761, 'direccion'] = 'Calle Mayor, 18'
rest1.loc[2823, 'direccion'] = 'Calle Castello, 3'
rest1.loc[2962, 'direccion'] = 'Calle Don Ramón de la Cruz, 109'
rest1.loc[3013, 'direccion'] = 'Calle de la Cava Baja, 23'
rest1.loc[3029, 'direccion'] = 'Calle Sor Ángela de la Cruz, 34'
rest1.loc[3049, 'direccion'] = 'Calle de Diego de León, 43'
rest1.loc[3062, 'direccion'] = 'Calle de Alcalá, 237'
rest1.loc[3118, 'direccion'] = 'Calle de Sevilla, 3'
rest1.loc[3241, 'direccion'] = 'Calle Gran Vía, 31'
rest1.loc[3300, 'direccion'] = 'Calle Urano, 1'
rest1.loc[3428, 'direccion'] = 'Calle Cardenal Cisneros, 15'
rest1.loc[3452, 'direccion'] = 'Calle de la Paz, 11'
rest1.loc[3525, 'direccion'] = 'Calle Covarrubias, 25'
rest1.loc[3543, 'direccion'] = 'Calle del Príncipe, 7'
rest1.loc[3640, 'direccion'] = 'Calle del Cardenal Cisneros, 3'
rest1.loc[3763, 'direccion'] = 'Avenida de la Gran Vía de Hortaleza, 57C'
rest1.loc[3806, 'direccion'] = 'Avenida General, 47'
rest1.loc[3877, 'direccion'] = 'Calle de Santa Engracia, 70'
rest1.loc[3896, 'direccion'] = 'Calle Almagro, 10'
rest1.loc[3896, 'cp'] = 28010
rest1.loc[4000, 'direccion'] = 'Calle Postigo de San Martín, 8'
rest1.loc[4111, 'direccion'] = 'Calle del Arroyo de Valdebebas, 10'
rest1.loc[4233, 'direccion'] = 'Calle de Padilla, 88'
rest1.loc[4294, 'direccion'] = 'Plaza Canovas del Castillo, 4'
rest1.loc[4315, 'direccion'] = 'Corredera Baja de San Pablo, 41'
rest1.loc[4344, 'direccion'] = 'Calle de la Paz, 11'
rest1.loc[6871, 'direccion'] = 'Avenida de la Hispanidad s/n'
rest1.loc[6871, 'cp'] = 28042
rest1.loc[4444, 'direccion'] = 'Calle Juan Alvarez Mendizábal, 4'
rest1.loc[4583, 'direccion'] = 'Avenida del Mediterráneo, 12'
rest1.loc[4673, 'direccion'] = 'Calle de Cea Bermúdez, 53'
rest1.loc[4754, 'direccion'] = 'Calle Medina de Pomar, 22'
rest1.loc[4773, 'direccion'] = 'Calle Montera, 36'
rest1.loc[4845, 'direccion'] = 'Calle de Villalar, 7'
rest1.loc[4956, 'direccion'] = 'Carrera de San Jeronimo 5'
rest1.loc[5401, 'direccion'] = 'Avenida de la Hispanidad, s/n'
rest1.loc[5401, 'cp'] = 28042
rest1.loc[5416, 'direccion'] = 'Calle Serrano, 61'
rest1.loc[5446, 'direccion'] = 'Calle de Blasco de Garay, 20'
rest1.loc[5667, 'direccion'] = 'Calle de Jesús, 6'
rest1.loc[5705, 'direccion'] = 'Calle Caleruega, 85'
rest1.loc[5724, 'direccion'] = 'Calle Villalar, 6'
rest1.loc[5733, 'direccion'] = 'Calle Santa Clara, 6'
rest1.loc[5853, 'direccion'] = 'Calle Gran Vía, 43'
rest1.loc[5979, 'direccion'] = 'Calle de Arturo Soria, 126'
rest1.loc[6219, 'direccion'] = 'Calle de Goya, 5'
rest1.loc[6286, 'direccion'] = 'Calle Marqués de Viana, 80'
rest1.loc[6300, 'direccion'] = 'Paseo de la Castellana, 71'
rest1.loc[6556, 'direccion'] = 'Calle Maestro Arbós, 3'
rest1.loc[6582, 'direccion'] = 'Avenida de Alberto Alcocer, 43'
rest1.loc[6634, 'direccion'] = 'Carretera Madrid - El Pardo, km 3.3'
rest1.loc[6699, 'direccion'] = 'Calle de Mendez Álvaro, 32C'
rest1.loc[6756, 'direccion'] = 'Calle Barco, 45'
rest1.loc[6896, 'direccion'] = 'Calle Arte Figurativo, 14'
rest1.loc[6915, 'direccion'] = 'Avenida de la Hispanidad s/n'
rest1.loc[6916, 'direccion'] = 'Calle Ferrocarril, 21'
rest1.loc[6954, 'direccion'] = 'Paseo de las Yeserías, 33'
rest1.loc[6984, 'direccion'] = 'Paseo de la Castellana, 266'
rest1.loc[6985, 'direccion'] = 'Calle de las Magnolias, 98'
rest1.loc[7006, 'direccion'] = 'Glorieta de Marqués de Vadillo, 4'
rest1.loc[7060, 'direccion'] = 'Calle de la Fe, 13'
rest1.loc[7292, 'direccion'] = 'Calle Espoz y Mina, 32'
rest1.loc[7313, 'direccion'] = 'Paseo del Prado, 36'
rest1.loc[7344, 'direccion'] = 'Calle Santiago, 10'
rest1.loc[7633, 'direccion'] = 'Calle Cava de San Miguel, 8'
rest1.loc[7635, 'direccion'] = 'Calle Hernani, 48'
rest1.loc[7657, 'direccion'] = 'Calle de la Princesa, 27'
rest1.loc[7813, 'direccion'] = 'Calle Montes de Toledo, 36'
rest1.loc[7872, 'direccion'] = 'Avenida de la Gran Vía del Sureste, 14'
rest1.loc[7932, 'direccion'] = 'Avenida Albufera, 119'
rest1.loc[7965, 'direccion'] = 'Calle del Príncipe, 14'
rest1.loc[8090, 'direccion'] = 'Carretera Madrid - El Pardo, km 1.4'
rest1.loc[8106, 'direccion'] = 'Calle de Melchor Fernández Almagro, 46'
rest1.loc[8157, 'direccion'] = 'Calle Calderilla, 1'
rest1.loc[8165, 'direccion'] = 'Avenida de Pablo Neruda, 91'
rest1.loc[7832, 'direccion'] = 'Calle del Monasterio de Arlanza, 20'
rest1.loc[8156, 'direccion'] = 'Glorieta Puerta de Toledo 4'
rest1.loc[8253, 'direccion'] = 'Paseo Imperial, 80'
rest1.loc[8400, 'direccion'] = 'Plaza de Isabel II, 7'
rest1.loc[8617, 'direccion'] = 'Calle Silva, 1'
rest1.loc[8674, 'direccion'] = 'Calle del Pez, 6'
rest1.loc[8730, 'direccion'] = 'Avenida del Manzanares, 210'
rest1.loc[8730, 'cp'] = 28026
rest1.loc[8731, 'direccion'] = 'Cuesta de Sto. Domingo, 24'
rest1.loc[8741, 'direccion'] = 'Calle Aragón, 8'
rest1.loc[8741, 'nombre'] = 'Restaurante Sakai'
rest1.loc[8820, 'direccion'] = 'Plaza del Emperador Carlos V'
rest1.loc[8893, 'direccion'] = 'Avenida de Pablo Neruda 91'
rest1.loc[8913, 'direccion'] = 'Calle López de Hoyos, 106'
rest1.loc[9046, 'direccion'] = 'Avenida de Pablo Neruda 91'
rest1.loc[9095, 'direccion'] = 'Calle de Seco, 1'
rest1.loc[9181, 'direccion'] = 'Plaza del Emperador Carlos V'
rest1.loc[9233, 'direccion'] = 'Calle de Arganda, 29'
rest1.loc[9251, 'direccion'] = 'Calle del Áncora, 26'
rest1.loc[9290, 'direccion'] = 'Calle Cerro de la Alcazaba, 36'
rest1.loc[9307, 'direccion'] = 'Calle Espoz y Mina, 32'
rest1.loc[9416, 'direccion'] =  'Plaza de Juan de Malasaña, 4'
rest1.loc[9471, 'direccion'] = 'Calle de Bravo Murillo, 122'
rest1.loc[9557, 'direccion'] = 'Paseo de la Castellana 40'
rest1.loc[248, 'direccion'] = 'Calle Senda del Infante, 28'
rest1.loc[332, 'direccion'] = 'Calle Jovellanos, 6'
rest1.loc[359, 'direccion'] = 'Calle de Santa Isabel, 5'
rest1.loc[628, 'direccion'] = 'Calle Libertad, 23'
rest1.loc[1031, 'direccion'] = 'Calle Ayala, 28'
rest1.loc[2544, 'direccion'] = 'Calle Valverde, 4'
rest1.loc[18,'direccion'] = 'Calle de Ventura de la Vega, 6'
rest1.loc[21,'direccion'] = 'Calle de Preciados, 11'
rest1.loc[26,'direccion'] = 'Calle Flor Baja, 6'

In [75]:
rest1.cp = rest1.cp.astype(str)

In [76]:
rest1 ['direccion_completa'] = (rest1.direccion + (', ') + rest1.cp + (', ') + rest1.ciudad)

In [77]:
lat = []
long = []

geolocator = Nominatim(user_agent="my_app")

for e in tqdm(rest1.direccion_completa):
    try:
        loc = geolocator.geocode(e)
        lat.append(loc.latitude)
        long.append(loc.longitude)
    except:
        lat.append('No disponible')
        long.append('No disponible')

100%|████████████████████████████████████████████████████████████████████████████| 9560/9560 [1:20:56<00:00,  1.97it/s]


In [20]:
rest1.insert(8, 'lat', lat)
rest1.insert(9, 'long', long)

In [67]:
rest1.loc[rest1.lat == 'No disponible']

,_id,nombre,puntuacion,direccion,cp,ciudad,pais,direccion_completa,lat,long,enlace,tipo_cocina
18,19,Indian Aroma,"4,5",Calle Ventura de La Vega 6,28014,Madrid,España,"Calle Ventura de La Vega 6, 28014, Madrid",No disponible,No disponible,http://www.restauranteindianaroma.com/,"['€€ - €€€', 'India', 'Asiática', 'Saludable']"
21,22,Oven Mozzarella,"5,0",Calle Preciados 11 Esquina Callejón De Preciados,28013,Madrid,España,Calle Preciados 11 Esquina Callejón De Preciad...,No disponible,No disponible,http://www.confuego.es/reservar-oven/reservar-...,"['Italiana', 'Pizza', 'Saludable', 'Del centro..."
26,27,GastroVía 61,"4,5","Calle Flor Baja, 3 Hotel Mayorazgo",28013,Madrid,España,"Calle Flor Baja, 3 Hotel Mayorazgo, 28013, Madrid",No disponible,No disponible,NaN,"['€€ - €€€', 'Mediterránea', 'Europea', 'Españ..."
27,28,Sibuya Urban Sushi Bar · CC Caleido,"5,0","Paseo de la Castellana, 259. 5th Tower",28046,Madrid,España,"Paseo de la Castellana, 259. 5th Tower, 28046,...",No disponible,No disponible,https://sibuyaurbansushibar.com/,[]
28,29,La Bistroteca,"5,0","Calle Espartinas 7, local izquierda Barrio Sal...",28001,Madrid,España,"Calle Espartinas 7, local izquierda Barrio Sal...",No disponible,No disponible,NaN,"['€€ - €€€', 'Fusión', 'Opciones vegetarianas'..."
29,30,Oven Mozzarella Gran Vía 55,"5,0","C / Gran Vía, 55 Oven, Steakburger y Pura Kaff...",28013,Madrid,España,"C / Gran Vía, 55 Oven, Steakburger y Pura Kaff...",No disponible,No disponible,http://www.oven.es/,"['€€ - €€€', 'Italiana', 'Pizza', 'Mediterránea']"
31,32,Johnny Rockets,"5,0",Calle Aracne S/n S/n,28022,Madrid,España,"Calle Aracne S/n S/n, 28022, Madrid",No disponible,No disponible,NaN,"['Americana', 'Comida rápida']"
33,34,Astral,"5,0",Camino Viejo de Leganés 82 Esquina Avenida de ...,28025,Madrid,España,Camino Viejo de Leganés 82 Esquina Avenida de ...,No disponible,No disponible,http://www.facebook.com/Astral-Cafeteria-16039...,"['€€ - €€€', 'Mediterránea', 'Europea', 'Españ..."
35,36,La Descarada,"5,0",73 Ortega y Gasset,28006,Madrid,España,"73 Ortega y Gasset, 28006, Madrid",No disponible,No disponible,http://www.ladescarada.es/,"['€€ - €€€', 'Mediterránea', 'Europea', 'Españ..."
39,40,La Vaca Argentina,"5,0",Calle Ventura De La Vega 3,28004,Madrid,España,"Calle Ventura De La Vega 3, 28004, Madrid",No disponible,No disponible,http://www.lavacaargentina.com/,"['€€€€', 'Restaurante de carne', 'Barbacoa', '..."


In [52]:
ls_prueba = list(df_prueba['direccion'])

In [61]:
num = []
no_num = []

for e in ls_prueba:

    if re.search(r'\d$', e):
        num.append(e)
    else:
        no_num.append(e)

In [64]:
num

['Calle Ventura de La Vega 6',
 'Camino Viejo de Leganés 82 Esquina Avenida de Oporto 53',
 'Calle Ventura De La Vega 3',
 'Calle de Cobos de Segovia, 30',
 'Gran Vía, 55 Galeria Gran Via 55',
 'Calle María Tubau 19',
 'Calle San Juan de Ortega No 36',
 'Calle Barcelona, 13 También acceso por Espoz y Mina 24',
 'Pº. Del Pintor Rosales 24',
 'Calle Bretón de los Herreros, 45 Local 2',
 'Calle General Diaz Porlier 95',
 'Menendez Pelayo 53 Local 2',
 'Calle Manuel Fernandez y Gonzalez 3',
 'Calle de la Princesa 13 Local 3',
 'Calle de Victoria 12',
 'Plaza Mayor 30 Entrada al restaurante por Calle de la Sal 2',
 'Plaza Independencia 5',
 'Calle de Hernán Córtez 19',
 'Calle PRINCIPE CARLOS, 21',
 'Calle San Bartolmé, 14',
 'Plaza Marina Española, 7',
 'Fernandez de los Rios, 93',
 'Calle Juan Alvarez Mendizabal 44',
 'Calle Lavapies 8 Calle Lavapies 8',
 'Calle Raimundo Fernández Villaverde 26',
 'Calle Cava Baja 30',
 'Calle de San Bernardo 5 Local 2',
 'Calle Caballero de Gracia,8 Call

In [66]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_app")

direccion_completa = 'Calle de Santa Engracia 103, 28010, Madrid'

location = geolocator.geocode(direccion_completa)

print((location.latitude, location.longitude))

(40.4370871, -3.6998872)


por ID:
    +8501 - eliminar
    +8583= lat: 40.4072266 lon: -3.6888640
    +
    

In [ ]:
try:
    location = geolocator.geocode("Avenida del Mediterráneo 12, 28007")
    if location:
        # Imprime la latitud y longitud si se obtuvo una ubicación válida
        print((location.latitude, location.longitude))
    else:
        print("No se pudo obtener la ubicación.")
except (GeocoderServiceError, GeocoderTimedOut) as e:
    print("Error al conectarse al servicio de geolocalización:", e)


In [ ]:
rest = rest.fillna('No hay link')

In [ ]:
nose = []
precio = []
precio1 = []
cocina = []


for e in rest1.tipo_cocina:
    nose.append(e.split(','))

for i in nose:
    if '€' in i[0]:
        precio.append(i[0])
        cocina.append(i[1:])
    else:
        precio.append('No hay información del precio')
        cocina.append(i[0:])

for s in precio:
    if "'" in s or '"' in s or '[]' in s:
        precio1.append(s.replace('"','').replace("'",'').replace('[',''))
    else:
        precio1.append('No hay información del precio')

In [ ]:
rest1['precio'] = precio1

In [ ]:
limpio = [e.copy() for e in cocina]

for i, e in enumerate(limpio):
    for j, el in enumerate(e):
        limpio[i][j] = el.replace('[','').replace(']','').replace("'",'').replace('"','').strip()

In [ ]:
rest1.tipo_cocina = limpio

In [ ]:
rest1.head()

In [ ]:
dummies = rest1['tipo_cocina'].apply(lambda x: pd.Series([1]*len(x), index=x)).fillna(0, downcast='infer')

In [ ]:
dummies.head()

In [ ]:
rest = pd.concat([rest, dummies], axis = 1)

In [ ]:
rest.head()